In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date


C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## S1 H2 adsorption isotherm

In [2]:
#read in the data
df= pd.read_csv("vaclog", sep='\t', index_col=False)
df.drop(columns=["I_emission","I_grid","T-platinum"], inplace=True)


In [3]:
#add elapsed time

timestamps=pd.to_datetime(df["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
df.insert(1,"Elapsed time",runtime)
df.head()


,Live comments,Elapsed time,Time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-CERNOX
0,NaN,0.0,05/12/2022 14:46:25,0.34736,9.250000e-08,9.190000e-08,5.192000e-08,4.779000e-07,0.000615,0.0,73.513
1,LHe transfer started,7.0,05/12/2022 14:46:32,0.34709,9.260000e-08,9.210000e-08,5.223000e-08,4.733000e-07,0.000598,0.0,74.074
2,NaN,15.0,05/12/2022 14:46:40,0.34679,9.260000e-08,9.210000e-08,5.235000e-08,4.688000e-07,0.000614,0.0,74.186
3,NaN,22.0,05/12/2022 14:46:47,0.34724,9.270000e-08,9.210000e-08,5.235000e-08,4.599000e-07,0.000605,0.0,74.523
4,NaN,29.0,05/12/2022 14:46:54,0.34758,9.280000e-08,9.230000e-08,5.235000e-08,4.582000e-07,0.000624,0.0,74.859


In [4]:
#Temperature curve for CERNOX 
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [5]:
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots()

ax.plot(df["Elapsed time"]/3600, polyfit(A,df["T-CERNOX"]),marker=".", markersize=5,label='Temperature')
ax.set_ylabel('Temperature (K)')

ax.set_xlabel('Elapsed time (h)')
ax2=ax.twinx()

ax2.plot((df["Elapsed time"]/3600),df['Barion_1'],marker=".", markersize=5,label='BA1')
ax2.plot((df["Elapsed time"]/3600),df['Barion_2'],marker=".", markersize=5,label='BA2')
ax2.plot((df["Elapsed time"]/3600),df['injection 100mbar'],marker=".", markersize=5,label='100mbar F-R')
ax2.set_ylabel('Pressure (mbar)')

plt.legend(loc="upper right")
plt.title("H2 adsorption isotherm measurement")
ax2.set_yscale('log')


In [6]:
#slicing the dataset
start = df.iloc[list(df["Elapsed time"]/3600<=1.85)]
ramp = df.iloc[list((df["Elapsed time"]/3600).between(1.85,40))]
print(ramp)

      Live comments  Elapsed time                 Time  injection 100mbar  \
927             NaN        6661.0  05/12/2022 16:37:26             80.558   
928             NaN        6668.0  05/12/2022 16:37:33             80.558   
929             NaN        6675.0  05/12/2022 16:37:40             80.559   
930             NaN        6682.0  05/12/2022 16:37:47             80.558   
931             NaN        6689.0  05/12/2022 16:37:54             80.559   
...             ...           ...                  ...                ...   
20028           NaN      143972.0  07/12/2022 06:45:57             78.276   
20029           NaN      143979.0  07/12/2022 06:46:04             78.278   
20030           NaN      143986.0  07/12/2022 06:46:11             78.279   
20031           NaN      143993.0  07/12/2022 06:46:18             78.279   
20032           NaN      144000.0  07/12/2022 06:46:25             78.280   

           Barion_2      Barion_1  DUAL experiment  DUAL insulation  \
927 

In [7]:
#plotting the sliced data
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots()

ax.plot((ramp["Elapsed time"]/3600),ramp['Barion_1'],marker=".", markersize=5,label='BA1')
ax.plot((ramp["Elapsed time"]/3600),ramp['Barion_2'],marker=".", markersize=5,label='BA2')
ax.plot((ramp["Elapsed time"]/3600),ramp['DUAL experiment'],marker=".", markersize=5,label='DUAL gauge')
plt.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')

plt.legend(loc="upper right")
plt.title("H2 adsorption isotherm close-up")


Text(0.5, 1.0, 'H2 adsorption isotherm close-up')